In [ ]:
# 1 Importações necessárias
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, month
from pyspark.ml.feature import StringIndexer, VectorAssembler, Normalizer, PCA
from pyspark.ml.regression import LinearRegression
from pyspark.sql.functions import lit
from pyspark.sql.types import IntegerType

# Inicializar a sessão Spark
spark = SparkSession.builder \
    .appName("Leitura Parquet") \
    .getOrCreate()

# Carregar o arquivo Parquet (substitua pelo caminho correto)
df_video = spark.read.parquet("C:/Users/diogo/Downloads/videos-comments-tratados.snappy.parquet")

# Mostrar as primeiras 10 linhas
df_video.show(10)


In [ ]:
# 2. Leia o arquivo Parquet
df_video = spark.read.parquet("C:/Users/diogo/Downloads/videos-comments-tratados.snappy.parquet")
print("Dados carregados:")
df_video.show(10)

In [ ]:
# 3. Adicione a coluna 'Month' com o valor do mês da coluna "Published At"
df_video = df_video.withColumn("Month", month(col("Published At")))
print("Coluna 'Month' adicionada:")
df_video.show(10)


In [ ]:
# 4. Adicione a coluna 'Keyword Index' transformando 'keyword' em valores numéricos

# Adicionar uma coluna de exemplo "keyword" com valor padrão
df_video = df_video.withColumn("keyword", lit("default_keyword"))

string_indexer = StringIndexer(inputCol="Keyword", outputCol="Keyword Index")

# Renomear a coluna (se necessário)
df_video = df_video.withColumnRenamed("Keyword com espaço", "keyword")

string_indexer = StringIndexer(inputCol="keyword", outputCol="Keyword Index")
df_video = string_indexer.fit(df_video).transform(df_video)
print("Coluna 'Keyword Index' adicionada:")
df_video.show(10)


In [ ]:
# 5. Crie o vetor "Features" com os campos: Likes, Views, Year, Month, Keyword Index
# Converter a coluna 'Year' para IntegerType
df_video = df_video.withColumn("Year", col("Year").cast(IntegerType()))

# Verificar se a conversão foi bem-sucedida
print("Coluna 'Year' convertida para IntegerType:")
df_video.printSchema()

# Criar o vetor "Features" novamente
feature_assembler = VectorAssembler(
    inputCols=["Likes", "Views", "Year", "Month", "Keyword Index"],
    outputCol="Features"
)

df_video = feature_assembler.transform(df_video)
print("Coluna 'Features' criada:")
df_video.show(10)



In [ ]:
# 6. Adicione a coluna 'Features Normal' com os dados normalizados
normalizer = Normalizer(inputCol="Features", outputCol="Features Normal", p=2.0)
df_video = normalizer.transform(df_video.na.drop(subset=["Features"]))
print("Coluna 'Features Normal' adicionada:")
df_video.show(10)


In [ ]:
# 7. Adicione a coluna 'Features PCA' com redução de características usando PCA
pca = PCA(k=1, inputCol="Features", outputCol="Features PCA")
pca_model = pca.fit(df_video)
df_video = pca_model.transform(df_video)
print("Coluna 'Features PCA' adicionada:")
df_video.show(10)

In [ ]:
# 8. Separe os dados em treinamento (80%) e teste (20%)
train_data, test_data = df_video.randomSplit([0.8, 0.2], seed=42)
print("Dados de treino:")
train_data.show(10)
print("Dados de teste:")
test_data.show(10)




In [ ]:
# 9. Crie e treine o modelo de regressão linear
lr = LinearRegression(featuresCol="Features Normal", labelCol="Comments")
lr_model = lr.fit(train_data)

# Avaliação do modelo
test_results = lr_model.evaluate(test_data)
print("R²: {:.4f}".format(test_results.r2))
print("RMSE: {:.4f}".format(test_results.rootMeanSquaredError))



In [ ]:
# Salvar o DataFrame em formato Parquet
df_video.write.mode("overwrite").parquet("videos-preparados-parquet")

# Arquivo Salvo
print("DataFrame salvo com sucesso em formato Parquet!")

# Finalizar a Spark Session
spark.stop()
